# scrape TMDb using api

In [1]:
from tmdbv3api import TMDb, Movie

In [2]:
apikey = '84f38901bc16f8f577112549b1e2878b'

In [3]:
tmdb = TMDb()
tmdb.api_key = apikey

In [4]:
tmdb.language = 'en'
tmdb.debug = True

In [5]:
movie = Movie()

In [10]:
len(movie.popular())

20

In [13]:
movie.popular()[1]

{'adult': False, 'backdrop_path': '/t5zCBSB5xMDKcDqe91qahCOUYVV.jpg', 'genre_ids': [27, 9648], 'id': 507089, 'original_language': 'en', 'original_title': "Five Nights at Freddy's", 'overview': "Recently fired and desperate for work, a troubled young man named Mike agrees to take a position as a night security guard at an abandoned theme restaurant: Freddy Fazbear's Pizzeria. But he soon discovers that nothing at Freddy's is what it seems.", 'popularity': 2817.844, 'poster_path': '/A4j8S6moJS2zNtRR8oWF08gRnL5.jpg', 'release_date': '2023-10-25', 'title': "Five Nights at Freddy's", 'video': False, 'vote_average': 8.061, 'vote_count': 2064}

# scrape IMDB using BeautifulSoup

In [2]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
url = 'https://www.imdb.com/chart/top/'
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}

In [24]:
response = requests.get(url, headers=header)
soup = BeautifulSoup(response.text, "html.parser")

In [26]:
movies = soup.find_all('li', class_='ipc-metadata-list-summary-item sc-59b6048d-0 cuaJSp cli-parent')

In [33]:
movie_1 = movies[0].find('a', class_='ipc-title-link-wrapper')['href']

In [46]:
movie_1

'/title/tt0111161/?ref_=chttp_t_1'

In [34]:
url_movie = 'https://www.imdb.com' + movie_1

In [43]:
response_movie = requests.get(url_movie, headers=header)
soup_movie = BeautifulSoup(response_movie.text, "html.parser")

In [44]:
name_1 = soup_movie.find('span', class_='sc-7f1a92f5-1 benbRT')

In [47]:
name_1.text

'Побег из Шоушенка'

# Scrape TMDb 

In [ ]:
base_url = 'https://api.themoviedb.org/3'
endpoint = '/discover/movie'
apikey = '?api_key=84f38901bc16f8f577112549b1e2878b'
query_params = '&language=en-US&sort_by=popularity.desc&include_adult=false&include_video=false&page='
numpage = 100

In [ ]:
base_url = 'https://api.themoviedb.org/3'
endpoint = '/genre/movie/list'
apikey = '?api_key=84f38901bc16f8f577112549b1e2878b'
query_params = '&language=en-US'

## Test scrape movies

In [205]:
url_genre = 'https://api.themoviedb.org/3/genre/movie/list?api_key=84f38901bc16f8f577112549b1e2878b&language=en-US'
response_genre = requests.get(url_genre)
data_genre = response_genre.json()
genres_name = data_genre['genres']

In [206]:
url = 'https://api.themoviedb.org/3/discover/movie?api_key=84f38901bc16f8f577112549b1e2878b&language=en-US&sort_by=popularity.desc&include_adult=false&include_video=false&page=2'
response = requests.get(url)
data = response.json()

In [207]:
movies = []
for movie in data["results"]:
    id = movie.get('id')
    # LAY THONG TIN TRAILER 
    url_trailer = 'https://api.themoviedb.org/3/movie/'+ str(id) + '/videos?api_key=84f38901bc16f8f577112549b1e2878b&language=en-US'
    response_trailer = requests.get(url_trailer)
    data_trailer = response_trailer.json()
    if data_trailer['results']:
        trailer_key = [video for video in data_trailer['results'] if video.get('type') == 'Trailer']
        trailer_link = 'https://www.youtube.com/embed/' + trailer_key[0]['key'] if trailer_key else ''
    else:
        trailer_link = ''

    # LAY THONG TIN CAST 
    url_cast = 'https://api.themoviedb.org/3/movie/' + str(id) + '/credits?api_key=84f38901bc16f8f577112549b1e2878b'
    response_cast = requests.get(url_cast)
    data_cast = response_cast.json()
    casts = []
    counter = 0
    for cast in data_cast['cast']:
        if cast.get('gender') == 2:
            gender = 'Male'
        elif cast.get('gender') == 1:
            gender = 'Female'
        else:
            gender = 'Other'
        name = cast.get('name')
        profile_path = 'https://www.themoviedb.org/t/p/w138_and_h175_face' + cast.get('profile_path') if cast.get('profile_path') else ''
        cast_id = cast.get('cast_id')
        casts.append({'gender':gender, 'name':name, 'profile_path':profile_path, 'cast_id':cast_id})
        if counter > 9:
            break
        counter += 1

    # LAY THONG TIN GENRES
    genre_ids = movie.get('genre_ids')
    genres = [{'id': genre['id'], 'name': genre['name']} for genre in genres_name if genre['id'] in genre_ids]

    # LAY THONG TIN KHAC 
    title = movie.get('title')
    overview = movie.get('overview')
    release_date = movie.get('release_date')    
    backdrop_path = 'https://www.themoviedb.org/t/p/w533_and_h300_bestv2' + movie.get('backdrop_path') if movie.get('backdrop_path') else ''
    original_language = movie.get('original_language')
    poster_path = 'https://www.themoviedb.org/t/p/w300_and_h450_bestv2' + movie.get('poster_path') if movie.get('poster_path') else ''
    vote_average = movie.get('vote_average')
    
    movies.append({'id':id,'title':title, 'overview':overview, 'genres':genres, 'release_date':release_date, 
                   'backdrop_path':backdrop_path, 'original_language':original_language, 'poster_path':poster_path,
                   'vote_average':vote_average, 'trailer_link':trailer_link, 'casts':casts})

df = pd.DataFrame(movies)
    


In [209]:
df.to_csv('tmdb_movie.csv')

## Test scrape genres

In [169]:
url = 'https://api.themoviedb.org/3/genre/movie/list?api_key=84f38901bc16f8f577112549b1e2878b&language=en-US'
response = requests.get(url)
data = response.json()

In [170]:
genres = []
for genre in data["genres"]:
    id = genre.get('id')
    name = genre.get('name')
    genres.append({'id':id, 'name':name})
df = pd.DataFrame(genres)

In [175]:
pd.DataFrame(data["genres"])

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [173]:
genres

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [172]:
genre_ids = [16, 35, 10751, 14, 10749]

In [171]:
df

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


## Test scrape trailer

In [141]:
url = 'https://api.themoviedb.org/3/movie/575264/videos?api_key=84f38901bc16f8f577112549b1e2878b&language=en-US'
response = requests.get(url)
data = response.json()

In [142]:
if data['results']:
    print('1')
else: 
    print('0')

1


In [143]:
trailers = [video for video in data['results'] if video.get('type') == 'Trailer']
trailers

[{'iso_639_1': 'en',
  'iso_3166_1': 'US',
  'name': 'Final Trailer',
  'key': 'HurjfO_TDlQ',
  'site': 'YouTube',
  'size': 1080,
  'type': 'Trailer',
  'official': True,
  'published_at': '2023-07-10T13:00:02.000Z',
  'id': '64ac6871e24b930100c731fb'},
 {'iso_639_1': 'en',
  'iso_3166_1': 'US',
  'name': 'Official Trailer',
  'key': 'avz06PDqDbM',
  'site': 'YouTube',
  'size': 1080,
  'type': 'Trailer',
  'official': True,
  'published_at': '2023-05-17T13:00:00.000Z',
  'id': '6464d96b02842001363889b6'},
 {'iso_639_1': 'en',
  'iso_3166_1': 'US',
  'name': 'Teaser Trailer with Intro by Tom Cruise',
  'key': '9YEH9IQwa_w',
  'site': 'YouTube',
  'size': 1080,
  'type': 'Trailer',
  'official': True,
  'published_at': '2022-09-07T19:06:47.000Z',
  'id': '6318ee8646e75f00a575619b'},
 {'iso_639_1': 'en',
  'iso_3166_1': 'US',
  'name': 'Official Teaser Trailer',
  'key': '2m1drlOZSDw',
  'site': 'YouTube',
  'size': 1080,
  'type': 'Trailer',
  'official': True,
  'published_at': '2022-

In [144]:
trailers[0]['key']

'HurjfO_TDlQ'

In [104]:
video_link = 'https://www.youtube.com/embed/' + trailers[0]['key']

In [105]:
video_link

'https://www.youtube.com/embed/HurjfO_TDlQ'

## test scrape cast 

In [177]:
url = 'https://api.themoviedb.org/3/movie/575264/credits?api_key=84f38901bc16f8f577112549b1e2878b'
response = requests.get(url)
data = response.json()

In [167]:
casts = []
for i in range(10):
    gender = data['cast'][i].get('gender')
    name = data['cast'][i].get('name')
    profile_path = 'https://www.themoviedb.org/t/p/w138_and_h175_face' + data['cast'][i].get('profile_path')
    cast_id = data['cast'][i].get('cast_id')
    casts.append({'gender':gender, 'name':name, 'profile_path':profile_path, 'cast_id':cast_id})

casts
    

[{'gender': 2,
  'name': 'Tom Cruise',
  'profile_path': 'https://www.themoviedb.org/t/p/w138_and_h175_face/eOh4ubpOm2Igdg0QH2ghj0mFtC.jpg',
  'cast_id': 4},
 {'gender': 1,
  'name': 'Hayley Atwell',
  'profile_path': 'https://www.themoviedb.org/t/p/w138_and_h175_face/cKUqIWH5Px237OW6LxsykR5dEE2.jpg',
  'cast_id': 7},
 {'gender': 2,
  'name': 'Ving Rhames',
  'profile_path': 'https://www.themoviedb.org/t/p/w138_and_h175_face/ohAOsD8E4tu35PI2buzZORpn9Ef.jpg',
  'cast_id': 14},
 {'gender': 2,
  'name': 'Simon Pegg',
  'profile_path': 'https://www.themoviedb.org/t/p/w138_and_h175_face/6ftW0kUeYno1kYfuwSTBILOtxjj.jpg',
  'cast_id': 18},
 {'gender': 1,
  'name': 'Rebecca Ferguson',
  'profile_path': 'https://www.themoviedb.org/t/p/w138_and_h175_face/lJloTOheuQSirSLXNA3JHsrMNfH.jpg',
  'cast_id': 11},
 {'gender': 1,
  'name': 'Vanessa Kirby',
  'profile_path': 'https://www.themoviedb.org/t/p/w138_and_h175_face/5fbvIkZ02RdcXfZHUUk4cQ9kILK.jpg',
  'cast_id': 15},
 {'gender': 2,
  'name': 'Esai

In [166]:
df

,gender,name,profile_path,cast_id
0,2,Tom Cruise,https://www.themoviedb.org/t/p/w138_and_h175_f...,4
1,1,Hayley Atwell,https://www.themoviedb.org/t/p/w138_and_h175_f...,7
2,2,Ving Rhames,https://www.themoviedb.org/t/p/w138_and_h175_f...,14
3,2,Simon Pegg,https://www.themoviedb.org/t/p/w138_and_h175_f...,18
4,1,Rebecca Ferguson,https://www.themoviedb.org/t/p/w138_and_h175_f...,11
5,1,Vanessa Kirby,https://www.themoviedb.org/t/p/w138_and_h175_f...,15
6,2,Esai Morales,https://www.themoviedb.org/t/p/w138_and_h175_f...,17
7,1,Pom Klementieff,https://www.themoviedb.org/t/p/w138_and_h175_f...,9
8,2,Henry Czerny,https://www.themoviedb.org/t/p/w138_and_h175_f...,13
9,2,Shea Whigham,https://www.themoviedb.org/t/p/w138_and_h175_f...,10


# Final scrape info movies

In [4]:
import requests
import csv
import pandas as pd

In [5]:
# get base_url page
url = 'https://api.themoviedb.org/3/discover/movie?api_key=84f38901bc16f8f577112549b1e2878b&language=en-US&sort_by=popularity.desc&page='
num_page = 499

In [6]:
movies = []

# loop each page
for i in range(num_page):

    # get page
    url_page = url + str(i+1)
    response_page = requests.get(url_page)
    data_page = response_page.json()
    
    # loop each movie
    for movie in data_page["results"]:
        id = movie.get('id')
        
        # go through movie and get details (trailer, casts, genres, ...)
        url_movie = 'https://api.themoviedb.org/3/movie/' + str(id) + '?api_key=84f38901bc16f8f577112549b1e2878b&append_to_response=videos,credits'
        response_movie = requests.get(url_movie)
        data_movie = response_movie.json()

        backdrop_path = 'https://www.themoviedb.org/t/p/w533_and_h300_bestv2' + data_movie.get('backdrop_path') if data_movie.get('backdrop_path') else ''

        budget = data_movie.get('budget')
        
        overview = data_movie.get('overview')
        popularity = data_movie.get('popularity')
        poster_path = 'https://www.themoviedb.org/t/p/w300_and_h450_bestv2' + data_movie.get('poster_path') if data_movie.get('poster_path') else ''
        release_date = data_movie.get('release_date')
        revenue = data_movie.get('revenue')
        runtime = data_movie.get('runtime')
        tagline = data_movie.get('tagline')
        title = data_movie.get('title')
        vote_average = data_movie.get('vote_average')
        vote_count = data_movie.get('vote_count')

        genres = data_movie.get('genres') 

        if data_movie.get('production_companies') != []: 
            production_companies = data_movie.get('production_companies')
        else:
            production_companies = [{"name": "None"}]

        if data_movie.get('production_countries') != []: 
            production_countries = data_movie.get('production_countries')
        else:
            production_countries = [{"name": "None"}]

        if data_movie.get('spoken_languages') != []: 
            spoken_languages = data_movie.get('spoken_languages')
        else:
            spoken_languages = [{"english_name": "None"}]

        # get trailer_link
        videos = data_movie.get('videos')
        if videos['results']:
            trailer_key = [video for video in videos['results'] if video.get('type') == 'Trailer']
            trailer_link = 'https://www.youtube.com/embed/' + trailer_key[0]['key'] if trailer_key else ''
        else:
            trailer_link = ''

        # get casts 
        credits = data_movie.get('credits')
        casts = []
        counter = 0
        for cast in credits['cast']:
            if cast.get('gender') == 2:
                gender = 'Male'
            elif cast.get('gender') == 1:
                gender = 'Female'
            else:
                gender = 'Other'
            name = cast.get('name')
            profile_path = 'https://www.themoviedb.org/t/p/w138_and_h175_face' + cast.get('profile_path') if cast.get('profile_path') else ''
            cast_id = cast.get('cast_id')
            casts.append({'gender':gender, 'name':name, 'profile_path':profile_path, 'cast_id':cast_id})
            if counter > 19:
                break
            counter += 1

        # append movie details into movies 
        movies.append({'backdrop_path':backdrop_path, 'budget':budget, 'genres':genres, 'overview':overview,
                       'popularity':popularity, 'poster_path':poster_path, 'production_companies':production_companies,
                       'production_countries':production_countries, 'release_date':release_date, 'revenue':revenue,
                       'runtime':runtime, 'spoken_languages':spoken_languages, 'tagline':tagline, 'title':title,
                       'vote_average':vote_average, 'vote_count':vote_count, 'trailer_link':trailer_link,
                       'casts':casts})
        
df = pd.DataFrame(movies)


KeyboardInterrupt: 

In [8]:
df

,backdrop_path,budget,genres,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,trailer_link,casts
0,https://www.themoviedb.org/t/p/w533_and_h300_b...,100000000,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",64 years before he becomes the tyrannical pres...,3233.513,https://www.themoviedb.org/t/p/w300_and_h450_b...,"[{'id': 1632, 'logo_path': '/cisLn1YAUuptXVBa0...","[{'iso_3166_1': 'US', 'name': 'United States o...",2023-11-15,315400000,157,"[{'english_name': 'English', 'iso_639_1': 'en'...",Everyone hungers for something.,The Hunger Games: The Ballad of Songbirds & Sn...,7.220,1024,https://www.youtube.com/embed/NxW_X4kzeus,"[{'gender': 'Male', 'name': 'Tom Blyth', 'prof..."
1,https://www.themoviedb.org/t/p/w533_and_h300_b...,205000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","Black Manta, still driven by the need to aveng...",1999.169,https://www.themoviedb.org/t/p/w300_and_h450_b...,"[{'id': 174, 'logo_path': '/9tefnXnJLGQL97cxs6...","[{'iso_3166_1': 'US', 'name': 'United States o...",2023-12-20,120000000,124,"[{'english_name': 'English', 'iso_639_1': 'en'...",The tide is turning.,Aquaman and the Lost Kingdom,6.549,184,https://www.youtube.com/embed/UGc5Tzz19UY,"[{'gender': 'Male', 'name': 'Jason Momoa', 'pr..."
2,https://www.themoviedb.org/t/p/w533_and_h300_b...,0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...","Dan Morgan is many things: a devoted husband, ...",1564.216,https://www.themoviedb.org/t/p/w300_and_h450_b...,"[{'id': 169668, 'logo_path': None, 'name': 'Mu...","[{'iso_3166_1': 'US', 'name': 'United States o...",2023-12-14,208,119,"[{'english_name': 'English', 'iso_639_1': 'en'...",,The Family Plan,7.496,355,https://www.youtube.com/embed/ns8weNznn1Y,"[{'gender': 'Male', 'name': 'Mark Wahlberg', '..."
3,https://www.themoviedb.org/t/p/w533_and_h300_b...,0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",A tormented father witnesses his young son die...,1329.943,https://www.themoviedb.org/t/p/w300_and_h450_b...,"[{'id': 3528, 'logo_path': '/cCzCClIzIh81Fa79h...","[{'iso_3166_1': 'US', 'name': 'United States o...",2023-11-30,0,104,"[{'english_name': 'No Language', 'iso_639_1': ...",Action speaks louder than words.,Silent Night,5.814,132,https://www.youtube.com/embed/yBnTqn0lBDA,"[{'gender': 'Male', 'name': 'Joel Kinnaman', '..."
4,https://www.themoviedb.org/t/p/w533_and_h300_b...,15000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...","After a Black Friday riot ends in tragedy, a m...",1126.478,https://www.themoviedb.org/t/p/w300_and_h450_b...,"[{'id': 175140, 'logo_path': '/grxF2M2ZByxjnzD...","[{'iso_3166_1': 'US', 'name': 'United States o...",2023-11-16,44000000,106,"[{'english_name': 'English', 'iso_639_1': 'en'...",There will be no leftovers.,Thanksgiving,6.650,274,https://www.youtube.com/embed/rc8vLvZ0fTE,"[{'gender': 'Male', 'name': 'Patrick Dempsey',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9975,https://www.themoviedb.org/t/p/w533_and_h300_b...,8000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",A clinically depressed teenager gets a new sta...,16.694,https://www.themoviedb.org/t/p/w300_and_h450_b...,"[{'id': 10146, 'logo_path': '/xnFIOeq5cKw09kCW...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-10-08,6491240,102,"[{'english_name': 'English', 'iso_639_1': 'en'...",Sometimes what's in your head isn't as crazy a...,It's Kind of a Funny Story,6.981,2231,https://www.youtube.com/embed/uXU3BDvq-sQ,"[{'gender': 'Male', 'name': 'Keir Gilchrist', ..."
9976,https://www.themoviedb.org/t/p/w533_and_h300_b...,8000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...","Friends since childhood, a magazine editor and...",16.694,https://www.themoviedb.org/t/p/w300_and_h450_b...,"[{'id': 43, 'logo_path': '/4RgIPr55kBakgupWkzd...","[{'iso_3166_1': 'US', 'name': 'United States o...",2002-10-05,27362712,109,"[{'english_name': 'English', 'i

In [ ]:
data = df 

In [ ]:
def convert(obj):
    L = []
    for i in obj:
        L.append(i['name'])
    return L

def convert_1(obj):
    L = []
    for i in obj:
        L.append(i['english_name'])
    return L

def convert_2(obj):
    L = []
    for i in obj:
        name = i.get('name')
        profile_path = i.get('profile_path', '')
        L.append({'name': name, 'profile_path': profile_path})
    return L

In [ ]:
data['genres'] = df['genres'].apply(convert)
data['production_companies'] = df['production_companies'].apply(convert)
data['production_countries'] = df['production_countries'].apply(convert)
data['spoken_languages'] = df['spoken_languages'].apply(convert_1)
data['casts'] = df['casts'].apply(convert_2)


In [ ]:
data[data['title']=='Night Train']

,backdrop_path,budget,genres,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,trailer_link,casts
156,https://www.themoviedb.org/t/p/w533_and_h300_b...,0,"[Action, Thriller, Crime]",Mother Holly McCord is driven to extremes to s...,165.757,https://www.themoviedb.org/t/p/w300_and_h450_b...,[None],[United States of America],2023-01-13,0,96,[English],She will do whatever it takes.,Night Train,6.28,25,https://www.youtube.com/embed/Eu31HgM5g6U,"[{'name': 'Danielle C. Ryan', 'profile_path': ..."


In [ ]:
data.to_csv('tmdb_movie.csv')

## test data

In [62]:
data = pd.read_csv('tmdb_movie.csv')

In [86]:
import json
import ast

In [90]:
def convert_to_list(json_str):
    try:
        return ast.literal_eval(json_str)
    except (ValueError, SyntaxError):
        return []

# Đọc dữ liệu từ tệp CSV và áp dụng chuyển đổi cho cột 'casts'
data = pd.read_csv('tmdb_movie.csv', converters={'casts': convert_to_list})

In [94]:
for index, row in data.iterrows():
    for cast in row['casts']:
        print(cast.get('name'))
        print(cast.get('profile_path',''))


Tom Cruise
https://www.themoviedb.org/t/p/w138_and_h175_face/eOh4ubpOm2Igdg0QH2ghj0mFtC.jpg
Hayley Atwell
https://www.themoviedb.org/t/p/w138_and_h175_face/cKUqIWH5Px237OW6LxsykR5dEE2.jpg
Ving Rhames
https://www.themoviedb.org/t/p/w138_and_h175_face/ohAOsD8E4tu35PI2buzZORpn9Ef.jpg
Simon Pegg
https://www.themoviedb.org/t/p/w138_and_h175_face/6ftW0kUeYno1kYfuwSTBILOtxjj.jpg
Rebecca Ferguson
https://www.themoviedb.org/t/p/w138_and_h175_face/lJloTOheuQSirSLXNA3JHsrMNfH.jpg
Vanessa Kirby
https://www.themoviedb.org/t/p/w138_and_h175_face/5fbvIkZ02RdcXfZHUUk4cQ9kILK.jpg
Esai Morales
https://www.themoviedb.org/t/p/w138_and_h175_face/kXQW3Uu8MnbIsPBzYNYDxFgDIDg.jpg
Pom Klementieff
https://www.themoviedb.org/t/p/w138_and_h175_face/hfUKAI2kXTMMWjno0i4sLPJud5N.jpg
Henry Czerny
https://www.themoviedb.org/t/p/w138_and_h175_face/zOWZzRmliKf9032IOOq0a3jXhVv.jpg
Shea Whigham
https://www.themoviedb.org/t/p/w138_and_h175_face/d3caK3l4UfbnzOxv95wLoFLZzMO.jpg
Greg Tarzan Davis
https://www.themoviedb.org/t/

In [95]:
data.shape

(100, 19)

In [4]:
A = [1, 2,3,4,5,6,7,8,9]
B = [1,2,3,3,3,2,5,6]
list(set(A) - set(B))

[8, 9, 4, 7]

SyntaxError: invalid syntax (1508878091.py, line 1)

In [3]:
a = [{'id': 1, 'name':'a'}, {'id': 2, 'name': 'b'}]
for dict in a:
    print(dict.get('id'))
    print(dict.get('name'))

1
a
2
b
